#Convert a shapefile of points to bounding boxes

**Before running this script, create a Google Drive folder with a shapefile of your point set (that would be overlaid on imagery). You should also know the approximate length of your target object in meters. The first half of the code only requires an input shapefile of points and outputs a shapefile of boxes. The second half of the code requires a tiling scheme file (`step 1`) and outputs those boxes as VIA annotations for those tiles, for use in model training.**

<a href="https://colab.research.google.com/github/gl7176/CNN_tools/blob/main/Points_to_VIA_annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
#####  <center> Be sure to update this hyperlink above if you clone and want to point to a different GitHub </center>

### Connect to our Google Drive folder and pull files
Note: when you run this it will give you a link that you must click. You must give Google some permissions, then copy a code into a box that comes up in the output section of this code.

If customizing this code, you will need to point the `drive_folder` variable to a URL for your shared google drive folder.

In [1]:
# set variable to the destination google drive folder you want to pull from
drive_folder = 'https://drive.google.com/drive/folders/1Dav7-r1nxziYVE0ouKaG2xHzr-da9-WF'

# enter approximate length of your object (here, a seal), in meters
# (this variable is used to draw the box around each point)
object_length = 2.6

!pip install -U -q PyDrive
import os, csv
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters

# this bit points the code to that google drive folder
pointer = str("'" + drive_folder.split("/")[-1] + "'" + " in parents")

file_list = drive.ListFile(
    {'q': pointer}).GetList()

#    this bit pulls key files from the directory specified above
#    and checks that all necessary files are present


for f in file_list:
  # 3. Create & download by id.
  fname = os.path.join(local_download_path, f['title'])
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
  print("Pulled file: " + fname)
  if fname.endswith(".shp"):
    ptfile = fname
  if fname.endswith(".json"):
    tiling_scheme_file = fname

Pulled file: data/tiling_scheme (4).json
Pulled file: data/Jbn01_3_03_20_L1A_Seals.shp
Pulled file: data/Jbn01_3_03_20_L1A_Seals.cpg
Pulled file: data/Jbn01_3_03_20_L1A_Seals.dbf
Pulled file: data/Jbn01_3_03_20_L1A_Seals.sbx
Pulled file: data/Jbn01_3_03_20_L1A_Seals.shx
Pulled file: data/Jbn01_3_03_20_L1A_Seals.sbn
Pulled file: data/Jbn01_3_03_20_L1A_Seals.shp.xml
Pulled file: data/Jbn01_3_03_20_L1A_Seals.prj


In [2]:
!pip install geopandas
!pip install affine
!pip install rasterio

     |████████████████████████████████| 1.0MB 28.5MB/s 
     |████████████████████████████████| 6.5MB 38.6MB/s 
     |████████████████████████████████| 15.3MB 224kB/s 
     |████████████████████████████████| 19.1MB 248kB/s 


### Draw boxes around each point

In [3]:
box_length = object_length

import fiona, ogr, numpy as np
from natsort import natsorted

box_list = []
with fiona.open(ptfile) as c:
    ptfile_crs = c.crs
    for i, record in enumerate(c):
        #import each point
        pt = ogr.Geometry(ogr.wkbPoint) 
        pt.SetPoint_2D(0, record["geometry"]["coordinates"][0], record["geometry"]["coordinates"][1])
        #buffer each point and draw "envelope" box around it
        buff = pt.Buffer(0.5*box_length)
        box = buff.GetEnvelope()
        bounding_box = [[box[0], box[2]], [box[1], box[2]], [box[1], box[3]], [box[0], box[3]]]
        # assemble shapefile for output
        entry = {"box":bounding_box}
        box_list.append(entry)

###Export boxes as shapefile, if wanted

In [4]:
output_dir = 'boxes_shapefile'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

import rasterio
from collections import OrderedDict
# Define your schema as a polygon geom with a couple of fields
schema = {
    'geometry': 'Polygon'
}

with fiona.open(output_dir + '/boxed_points.shp',
    'w',
    driver='ESRI Shapefile',
    crs=ptfile_crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(box_list):
      record = {
            'geometry': {'coordinates': [polygon['box']], 'type': 'Polygon'},
            'id': num,
            'type': 'Feature'}
      c.write(record)

# zip up the output directory into an archive for download
output_file_name = 'shapefile_outputs'
import subprocess
subprocess.call(['zip', '-r', output_file_name + '.zip', '/content/' + output_dir])

from google.colab import files
files.download(output_file_name + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Import tiling scheme information and set up variables and functions for coordinate conversions

In [11]:
import json
from affine import Affine

with open(tiling_scheme_file) as f:
    tiling_scheme = json.load(f)
    tile_height = tiling_scheme["tile_height"]
    tile_width = tiling_scheme["tile_width"]
    tile_overlap = tiling_scheme["tile_overlap"]
    proj = tiling_scheme["spatial_reference"]
    gt = tiling_scheme["transform"]
    img_data = tiling_scheme["tile_pointers"]

geotransform = (gt[2], gt[0], gt[1], gt[5], gt[3], gt[4])
geotransform = Affine.from_gdal(*geotransform)

x_tile_dist = tile_width - tile_overlap
y_tile_dist = tile_height - tile_overlap

def locate_in_tile(bounding_box):
  entry = []
  lx_corner, lx_remainder = divmod(bounding_box[3][0], x_tile_dist)
  ty_corner, ty_remainder = divmod(bounding_box[3][1], y_tile_dist)
  rx_remainder = bounding_box[1][0] - (lx_corner*x_tile_dist)
  by_remainder = bounding_box[1][1] - (ty_corner*y_tile_dist)
  box_dimension = rx_remainder - lx_remainder
  tl_tile_point = "[{x}, {y}]".format(x=int(lx_corner*x_tile_dist), y=int(ty_corner*y_tile_dist))
  inv_map = {str(v): k for k, v in img_data['image_locations'].items()}
  # this variable determines how "clipped" an edge box can be before we throw it away
  disregard_threshold = 0.5
  if rx_remainder < tile_width+disregard_threshold*box_dimension and by_remainder < tile_height+disregard_threshold*box_dimension:
    tile_info = inv_map[tl_tile_point], int(lx_corner*x_tile_dist), int(ty_corner*y_tile_dist)
    entry.append(tile_info)
  # use remainder to determine whether a detection occurs in overlap and needs to be multiple-annotated
  new_lx_corner = lx_corner
  new_ty_corner = ty_corner
  if rx_remainder < tile_overlap+disregard_threshold*box_dimension:
    new_lx_corner = lx_corner-1
    #print("left margin")
  elif lx_remainder > tile_width-(tile_overlap+disregard_threshold*box_dimension):
    new_lx_corner = lx_corner+1
    #print("right margin")
  if by_remainder < tile_overlap+disregard_threshold*box_dimension:
    new_ty_corner = ty_corner-1
    #print("top margin")
  elif ty_remainder > tile_height-(tile_overlap+disregard_threshold*box_dimension):
    new_ty_corner = ty_corner+1
    #print("bottom margin")
  if new_lx_corner != lx_corner:
    tl_tile_point = "[{x}, {y}]".format(x=int(new_lx_corner*x_tile_dist), y=int(ty_corner*y_tile_dist))
    try:
      tile_info = inv_map[tl_tile_point], int(new_lx_corner*x_tile_dist), int(ty_corner*y_tile_dist)
      entry.append(tile_info)
    except:
      print("adjacent tile does not exist")
  if new_ty_corner != ty_corner:
    tl_tile_point = "[{x}, {y}]".format(x=int(lx_corner*x_tile_dist), y=int(new_ty_corner*y_tile_dist))
    try:
      tile_info = inv_map[tl_tile_point], int(lx_corner*x_tile_dist), int(new_ty_corner*y_tile_dist)
      entry.append(tile_info)
    except:
      print("adjacent tile does not exist")
  if new_lx_corner != lx_corner and new_ty_corner != ty_corner:
    #print("double margin")
    tl_tile_point = "[{x}, {y}]".format(x=int(new_lx_corner*x_tile_dist), y=int(new_ty_corner*y_tile_dist))
    try:
      tile_info = inv_map[tl_tile_point], int(new_lx_corner*x_tile_dist), int(new_ty_corner*y_tile_dist)
      entry.append(tile_info)
    except:
      print("bishop tile does not exist")
  return entry

### Convert boxes from global coordinates to tile coordinates

In [13]:
entry_list = []
for value in box_list:
  bounding_box = value['box']
  # transform box coordinates from global to orthomosaic
  new_box = []
  for k, point in enumerate(bounding_box):
    #https://www.perrygeo.com/python-affine-transforms.html
    new_box.append(~geotransform * point)
  #transform box coodinates from orthomosaic to tile
  for q in locate_in_tile(new_box):
    tile_ID, x_offset, y_offset = q
    newer_box = []
    for point in new_box:
      newer_box.append((point[0]-x_offset, point[1]-y_offset))
  # reassemble boxes entries with tile name and coordinates
    entry = {"tile_ID":tile_ID, "box": newer_box}
    entry_list.append(entry)
# sort boxes by tile name
def SortFunc(e):
  return e['tile_ID']
entry_list = natsorted(entry_list, key=SortFunc)

### Output detections in VIA format

In [14]:
# add class info later, when we have it on-hand to work with
# class_category = "Age Class"

#2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---28.png,1613979,"{}",4,0,"{""name"":""rect"",""x"":615,""y"":927,""width"":66,""height"":32}","{""Age Class"":""Adult""}"
#2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---28.png,1613979,"{}",4,1,"{""name"":""rect"",""x"":959,""y"":917,""width"":39,""height"":26}","{""Age Class"":""Pup""}"
new_line = [["filename","file_size","file_attributes","region_count","region_id","region_shape_attributes","region_attributes"]]

filename = ""
for detection in entry_list:
    print(detection["box"])
    temp = []
    if filename != detection["tile_ID"]:
      filename = detection["tile_ID"]
      count = 0
    else:
      count += 1
    file_size = ""
    file_attributes = "{}"
    x1 = detection["box"][3][0]
    y1 = detection["box"][3][1]
    x2 = detection["box"][1][0]
    y2 = detection["box"][1][1]
    #print("x1={x1}, x2={x2}, y1={y1}, y2={y2}".format(x1=x1,x2=x2,y1=y1,y2=y2))
    region_shape_attributes = {"name":"rect", "x":x1, "y":y1, "width":x2-x1, "height":y2-y1}
    region_count = ""
    region_attributes = {}
    region_ID = count
    new_line.append([filename, file_size, file_attributes, region_count, region_ID, region_shape_attributes, region_attributes])

for k, x in enumerate(new_line):
  new_line[k][5],new_line[k][6] = str(x[5]).replace("'",'"'),str(x[6]).replace("'",'"')

# Set output directory, create it if necessary
output_dir = 'via_annotations'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# write out new VIA file with additional detections
with open(output_dir + '/new_VIA_annotations.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(new_line)

from google.colab import files
files.download(output_dir + '/new_VIA_annotations.csv')

[(1203.156595993787, 1925.40767917037), (1370.7903806455433, 1925.40767917037), (1370.7903806455433, 1757.7738945186138), (1203.156595993787, 1757.7738945186138)]
[(716.7321232520044, 1931.0902080833912), (884.3659079037607, 1931.0902080833912), (884.3659079037607, 1763.456423431635), (716.7321232520044, 1763.456423431635)]
[(389.41845934838057, 2051.559820473194), (557.0522440038621, 2051.559820473194), (557.0522440038621, 1883.9260358214378), (389.41845934838057, 1883.9260358214378)]
[(815.8640089891851, 590.4659841358662), (983.4977936446667, 590.4659841358662), (983.4977936446667, 422.8321994841099), (815.8640089891851, 422.8321994841099)]
[(1233.916317526251, 847.1603074073792), (1401.5501021780074, 847.1603074073792), (1401.5501021780074, 679.5265227556229), (1233.916317526251, 679.5265227556229)]
[(1373.6850888803601, 1212.0303631424904), (1541.3188735321164, 1212.0303631424904), (1541.3188735321164, 1044.396578490734), (1373.6850888803601, 1044.396578490734)]
[(1348.67383505776

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>